# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [7]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [5]:
# Preview the data
print(X_train.shape)
X_train.head()

(1051, 296)


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [8]:
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation = 'relu', input_shape = (n_features,)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation = 'relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation = 'linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 195us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 2/150
1051/1051 [==============================] - 0s 73us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 3/150
1051/1051 [==============================] - 0s 82us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 4/150
1051/1051 [==============================] - 0s 72us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 5/150
1051/1051 [==============================] - 0s 83us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 6/150
1051/1051 [==============================] - 0s 88us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 7/150
1051/1051 [=====

1051/1051 [==============================] - 0s 73us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 54/150
1051/1051 [==============================] - 0s 69us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 55/150
1051/1051 [==============================] - 0s 66us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 56/150
1051/1051 [==============================] - 0s 68us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 57/150
1051/1051 [==============================] - 0s 65us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 58/150
1051/1051 [==============================] - 0s 65us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 59/150
1051/1051 [==============================] - 0s 66us/step - loss: nan 

Epoch 106/150
1051/1051 [==============================] - 0s 67us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 107/150
1051/1051 [==============================] - 0s 67us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 108/150
1051/1051 [==============================] - 0s 66us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 109/150
1051/1051 [==============================] - 0s 63us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 110/150
1051/1051 [==============================] - 0s 63us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 111/150
1051/1051 [==============================] - 0s 74us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 112/150
1051/1051 [==============================] - 0s 86

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [12]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns = numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns = numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns = numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [13]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [14]:
# Train the model 
normalized_input_model.fit(X_train, y_train, epochs = 150, batch_size = 32, validation_data = (X_val, y_val))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 209us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 2/150
1051/1051 [==============================] - 0s 83us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 3/150
1051/1051 [==============================] - 0s 73us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 4/150
1051/1051 [==============================] - 0s 81us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 5/150
1051/1051 [==============================] - 0s 68us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 6/150
1051/1051 [==============================] - 0s 77us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 7/150
1051/1051 [=====

1051/1051 [==============================] - 0s 71us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 54/150
1051/1051 [==============================] - 0s 72us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 55/150
1051/1051 [==============================] - 0s 67us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 56/150
1051/1051 [==============================] - 0s 67us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 57/150
1051/1051 [==============================] - 0s 65us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 58/150
1051/1051 [==============================] - 0s 66us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 59/150
1051/1051 [==============================] - 0s 69us/step - loss: nan 

Epoch 106/150
1051/1051 [==============================] - 0s 77us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 107/150
1051/1051 [==============================] - 0s 71us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 108/150
1051/1051 [==============================] - 0s 69us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 109/150
1051/1051 [==============================] - 0s 75us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 110/150
1051/1051 [==============================] - 0s 77us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 111/150
1051/1051 [==============================] - 0s 74us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 112/150
1051/1051 [==============================] - 0s 65

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [15]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [16]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, y_train_scaled, epochs = 150, batch_size = 32, 
                     validation_data = (X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 215us/step - loss: 0.4368 - mean_squared_error: 0.4368 - val_loss: 0.1992 - val_mean_squared_error: 0.1992
Epoch 2/150
1051/1051 [==============================] - 0s 71us/step - loss: 0.2274 - mean_squared_error: 0.2274 - val_loss: 0.1613 - val_mean_squared_error: 0.1613
Epoch 3/150
1051/1051 [==============================] - 0s 71us/step - loss: 0.1842 - mean_squared_error: 0.1842 - val_loss: 0.1588 - val_mean_squared_error: 0.1588
Epoch 4/150
1051/1051 [==============================] - 0s 67us/step - loss: 0.1642 - mean_squared_error: 0.1642 - val_loss: 0.1376 - val_mean_squared_error: 0.1376
Epoch 5/150
1051/1051 [==============================] - 0s 74us/step - loss: 0.1480 - mean_squared_error: 0.1480 - val_loss: 0.1321 - val_mean_squared_error: 0.1321
Epoch 6/150
1051/1051 [==============================] - 0s 69us/step - loss: 0.1330 - mean_squared_error: 0.1330 - val_lo

Epoch 50/150
1051/1051 [==============================] - 0s 65us/step - loss: 0.0317 - mean_squared_error: 0.0317 - val_loss: 0.1019 - val_mean_squared_error: 0.1019
Epoch 51/150
1051/1051 [==============================] - 0s 64us/step - loss: 0.0316 - mean_squared_error: 0.0316 - val_loss: 0.1032 - val_mean_squared_error: 0.1032
Epoch 52/150
1051/1051 [==============================] - 0s 64us/step - loss: 0.0308 - mean_squared_error: 0.0308 - val_loss: 0.1033 - val_mean_squared_error: 0.1033
Epoch 53/150
1051/1051 [==============================] - 0s 66us/step - loss: 0.0301 - mean_squared_error: 0.0301 - val_loss: 0.1034 - val_mean_squared_error: 0.1034
Epoch 54/150
1051/1051 [==============================] - 0s 67us/step - loss: 0.0290 - mean_squared_error: 0.0290 - val_loss: 0.1048 - val_mean_squared_error: 0.1048
Epoch 55/150
1051/1051 [==============================] - 0s 66us/step - loss: 0.0285 - mean_squared_error: 0.0285 - val_loss: 0.1038 - val_mean_squared_error: 0.103

1051/1051 [==============================] - 0s 75us/step - loss: 0.0143 - mean_squared_error: 0.0143 - val_loss: 0.1129 - val_mean_squared_error: 0.1129
Epoch 100/150
1051/1051 [==============================] - 0s 69us/step - loss: 0.0141 - mean_squared_error: 0.0141 - val_loss: 0.1121 - val_mean_squared_error: 0.1121
Epoch 101/150
1051/1051 [==============================] - 0s 71us/step - loss: 0.0140 - mean_squared_error: 0.0140 - val_loss: 0.1134 - val_mean_squared_error: 0.1134
Epoch 102/150
1051/1051 [==============================] - 0s 73us/step - loss: 0.0137 - mean_squared_error: 0.0137 - val_loss: 0.1127 - val_mean_squared_error: 0.1127
Epoch 103/150
1051/1051 [==============================] - 0s 80us/step - loss: 0.0136 - mean_squared_error: 0.0136 - val_loss: 0.1131 - val_mean_squared_error: 0.1131
Epoch 104/150
1051/1051 [==============================] - 0s 71us/step - loss: 0.0133 - mean_squared_error: 0.0133 - val_loss: 0.1136 - val_mean_squared_error: 0.1136
Epoch 

Epoch 148/150
1051/1051 [==============================] - 0s 67us/step - loss: 0.0082 - mean_squared_error: 0.0082 - val_loss: 0.1185 - val_mean_squared_error: 0.1185
Epoch 149/150
1051/1051 [==============================] - 0s 68us/step - loss: 0.0081 - mean_squared_error: 0.0081 - val_loss: 0.1182 - val_mean_squared_error: 0.1182
Epoch 150/150
1051/1051 [==============================] - 0s 66us/step - loss: 0.0082 - mean_squared_error: 0.0082 - val_loss: 0.1194 - val_mean_squared_error: 0.1194


Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [17]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 30us/step


[0.007801741426748593, 0.007801741426748593]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [18]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 39us/step


[0.11936161760594909, 0.11936161760594909]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [19]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

27149.626295847014

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [20]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, activation = 'relu', input_shape = (n_features,), 
                          kernel_initializer = 'he_normal'))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 234us/step - loss: 0.5083 - mean_squared_error: 0.5083 - val_loss: 0.1699 - val_mean_squared_error: 0.1699
Epoch 2/150
1051/1051 [==============================] - 0s 76us/step - loss: 0.2217 - mean_squared_error: 0.2217 - val_loss: 0.1432 - val_mean_squared_error: 0.1432
Epoch 3/150
1051/1051 [==============================] - 0s 72us/step - loss: 0.1798 - mean_squared_error: 0.1798 - val_loss: 0.1378 - val_mean_squared_error: 0.1378
Epoch 4/150
1051/1051 [==============================] - 0s 73us/step - loss: 0.1569 - mean_squared_error: 0.1569 - val_loss: 0.1250 - val_mean_squared_error: 0.1250
Epoch 5/150
1051/1051 [==============================] - 0s 73us/step - loss: 0.1411 - mean_squared_error: 0.1411 - val_loss: 0.1196 - val_mean_squared_error: 0.1196
Epoch 6/150
1051/1051 [==============================] - 0s 76us/step - loss: 0.1265 - mean_squared_error: 0.1265 - val_lo

Epoch 50/150
1051/1051 [==============================] - 0s 75us/step - loss: 0.0303 - mean_squared_error: 0.0303 - val_loss: 0.1092 - val_mean_squared_error: 0.1092
Epoch 51/150
1051/1051 [==============================] - 0s 73us/step - loss: 0.0296 - mean_squared_error: 0.0296 - val_loss: 0.1094 - val_mean_squared_error: 0.1094
Epoch 52/150
1051/1051 [==============================] - 0s 64us/step - loss: 0.0293 - mean_squared_error: 0.0293 - val_loss: 0.1104 - val_mean_squared_error: 0.1104
Epoch 53/150
1051/1051 [==============================] - 0s 84us/step - loss: 0.0287 - mean_squared_error: 0.0287 - val_loss: 0.1105 - val_mean_squared_error: 0.1105
Epoch 54/150
1051/1051 [==============================] - 0s 76us/step - loss: 0.0281 - mean_squared_error: 0.0281 - val_loss: 0.1109 - val_mean_squared_error: 0.1109
Epoch 55/150
1051/1051 [==============================] - 0s 68us/step - loss: 0.0278 - mean_squared_error: 0.0278 - val_loss: 0.1104 - val_mean_squared_error: 0.110

1051/1051 [==============================] - 0s 71us/step - loss: 0.0144 - mean_squared_error: 0.0144 - val_loss: 0.1194 - val_mean_squared_error: 0.1194
Epoch 100/150
1051/1051 [==============================] - 0s 71us/step - loss: 0.0142 - mean_squared_error: 0.0142 - val_loss: 0.1190 - val_mean_squared_error: 0.1190
Epoch 101/150
1051/1051 [==============================] - 0s 70us/step - loss: 0.0140 - mean_squared_error: 0.0140 - val_loss: 0.1189 - val_mean_squared_error: 0.1189
Epoch 102/150
1051/1051 [==============================] - 0s 66us/step - loss: 0.0140 - mean_squared_error: 0.0140 - val_loss: 0.1182 - val_mean_squared_error: 0.1182
Epoch 103/150
1051/1051 [==============================] - 0s 69us/step - loss: 0.0139 - mean_squared_error: 0.0139 - val_loss: 0.1183 - val_mean_squared_error: 0.1183
Epoch 104/150
1051/1051 [==============================] - 0s 69us/step - loss: 0.0136 - mean_squared_error: 0.0136 - val_loss: 0.1206 - val_mean_squared_error: 0.1206
Epoch 

Epoch 148/150
1051/1051 [==============================] - 0s 66us/step - loss: 0.0086 - mean_squared_error: 0.0086 - val_loss: 0.1246 - val_mean_squared_error: 0.1246
Epoch 149/150
1051/1051 [==============================] - 0s 65us/step - loss: 0.0085 - mean_squared_error: 0.0085 - val_loss: 0.1243 - val_mean_squared_error: 0.1243
Epoch 150/150
1051/1051 [==============================] - 0s 64us/step - loss: 0.0084 - mean_squared_error: 0.0084 - val_loss: 0.1249 - val_mean_squared_error: 0.1249


Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [22]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 30us/step


[0.008191877647305704, 0.008191877647305704]

Evaluate the model (`he_model`) on validate data (`X_train` and `y_train_scaled`) 

In [24]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 52us/step


[0.12485781824747419, 0.12485781824747419]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [27]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=(n_features,)))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 249us/step - loss: 0.4769 - mean_squared_error: 0.4769 - val_loss: 0.1461 - val_mean_squared_error: 0.1461
Epoch 2/150
1051/1051 [==============================] - 0s 66us/step - loss: 0.2096 - mean_squared_error: 0.2096 - val_loss: 0.1229 - val_mean_squared_error: 0.1229
Epoch 3/150
1051/1051 [==============================] - 0s 74us/step - loss: 0.1699 - mean_squared_error: 0.1699 - val_loss: 0.1167 - val_mean_squared_error: 0.1167
Epoch 4/150
1051/1051 [==============================] - 0s 72us/step - loss: 0.1479 - mean_squared_error: 0.1479 - val_loss: 0.1085 - val_mean_squared_error: 0.1085
Epoch 5/150
1051/1051 [==============================] - 0s 74us/step - loss: 0.1331 - mean_squared_error: 0.1331 - val_loss: 0.1059 - val_mean_squared_error: 0.1059
Epoch 6/150
1051/1051 [==============================] - 0s 73us/step - loss: 0.1197 - mean_squared_error: 0.1197 - val_lo

Epoch 50/150
1051/1051 [==============================] - 0s 75us/step - loss: 0.0267 - mean_squared_error: 0.0267 - val_loss: 0.1159 - val_mean_squared_error: 0.1159
Epoch 51/150
1051/1051 [==============================] - 0s 69us/step - loss: 0.0261 - mean_squared_error: 0.0261 - val_loss: 0.1174 - val_mean_squared_error: 0.1174
Epoch 52/150
1051/1051 [==============================] - 0s 68us/step - loss: 0.0259 - mean_squared_error: 0.0259 - val_loss: 0.1190 - val_mean_squared_error: 0.1190
Epoch 53/150
1051/1051 [==============================] - 0s 68us/step - loss: 0.0253 - mean_squared_error: 0.0253 - val_loss: 0.1194 - val_mean_squared_error: 0.1194
Epoch 54/150
1051/1051 [==============================] - 0s 65us/step - loss: 0.0247 - mean_squared_error: 0.0247 - val_loss: 0.1191 - val_mean_squared_error: 0.1191
Epoch 55/150
1051/1051 [==============================] - 0s 73us/step - loss: 0.0244 - mean_squared_error: 0.0244 - val_loss: 0.1188 - val_mean_squared_error: 0.118

1051/1051 [==============================] - 0s 70us/step - loss: 0.0126 - mean_squared_error: 0.0126 - val_loss: 0.1292 - val_mean_squared_error: 0.1292
Epoch 100/150
1051/1051 [==============================] - 0s 66us/step - loss: 0.0124 - mean_squared_error: 0.0124 - val_loss: 0.1288 - val_mean_squared_error: 0.1288
Epoch 101/150
1051/1051 [==============================] - 0s 67us/step - loss: 0.0124 - mean_squared_error: 0.0124 - val_loss: 0.1293 - val_mean_squared_error: 0.1293
Epoch 102/150
1051/1051 [==============================] - 0s 71us/step - loss: 0.0123 - mean_squared_error: 0.0123 - val_loss: 0.1276 - val_mean_squared_error: 0.1276
Epoch 103/150
1051/1051 [==============================] - 0s 65us/step - loss: 0.0120 - mean_squared_error: 0.0120 - val_loss: 0.1283 - val_mean_squared_error: 0.1283
Epoch 104/150
1051/1051 [==============================] - 0s 72us/step - loss: 0.0119 - mean_squared_error: 0.0119 - val_loss: 0.1300 - val_mean_squared_error: 0.1300
Epoch 

Epoch 148/150
1051/1051 [==============================] - 0s 71us/step - loss: 0.0072 - mean_squared_error: 0.0072 - val_loss: 0.1353 - val_mean_squared_error: 0.1353
Epoch 149/150
1051/1051 [==============================] - 0s 70us/step - loss: 0.0072 - mean_squared_error: 0.0072 - val_loss: 0.1336 - val_mean_squared_error: 0.1336
Epoch 150/150
1051/1051 [==============================] - 0s 67us/step - loss: 0.0072 - mean_squared_error: 0.0072 - val_loss: 0.1352 - val_mean_squared_error: 0.1352


Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [28]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 32us/step


[0.007074010766666918, 0.007074010766666918]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [29]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 41us/step


[0.13515970881447592, 0.13515970881447592]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [30]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer = 'rmsprop', loss = 'mse', metrics = ['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 305us/step - loss: 0.3204 - mean_squared_error: 0.3204 - val_loss: 0.1367 - val_mean_squared_error: 0.1367
Epoch 2/150
1051/1051 [==============================] - 0s 81us/step - loss: 0.1667 - mean_squared_error: 0.1667 - val_loss: 0.1119 - val_mean_squared_error: 0.1119
Epoch 3/150
1051/1051 [==============================] - 0s 87us/step - loss: 0.1241 - mean_squared_error: 0.1241 - val_loss: 0.1289 - val_mean_squared_error: 0.1289
Epoch 4/150
1051/1051 [==============================] - 0s 71us/step - loss: 0.0992 - mean_squared_error: 0.0992 - val_loss: 0.1103 - val_mean_squared_error: 0.1103
Epoch 5/150
1051/1051 [==============================] - 0s 72us/step - loss: 0.0891 - mean_squared_error: 0.0891 - val_loss: 0.1027 - val_mean_squared_error: 0.1027
Epoch 6/150
1051/1051 [==============================] - 0s 79us/step - loss: 0.0670 - mean_squared_error: 0.0670 - val_lo

Epoch 50/150
1051/1051 [==============================] - 0s 81us/step - loss: 0.0088 - mean_squared_error: 0.0088 - val_loss: 0.0972 - val_mean_squared_error: 0.0972
Epoch 51/150
1051/1051 [==============================] - 0s 75us/step - loss: 0.0100 - mean_squared_error: 0.0100 - val_loss: 0.0962 - val_mean_squared_error: 0.0962
Epoch 52/150
1051/1051 [==============================] - 0s 71us/step - loss: 0.0081 - mean_squared_error: 0.0081 - val_loss: 0.0920 - val_mean_squared_error: 0.0920
Epoch 53/150
1051/1051 [==============================] - 0s 74us/step - loss: 0.0083 - mean_squared_error: 0.0083 - val_loss: 0.0922 - val_mean_squared_error: 0.0922
Epoch 54/150
1051/1051 [==============================] - 0s 72us/step - loss: 0.0081 - mean_squared_error: 0.0081 - val_loss: 0.0904 - val_mean_squared_error: 0.0904
Epoch 55/150
1051/1051 [==============================] - 0s 70us/step - loss: 0.0071 - mean_squared_error: 0.0071 - val_loss: 0.0884 - val_mean_squared_error: 0.088

1051/1051 [==============================] - 0s 70us/step - loss: 0.0058 - mean_squared_error: 0.0058 - val_loss: 0.1055 - val_mean_squared_error: 0.1055
Epoch 100/150
1051/1051 [==============================] - 0s 71us/step - loss: 0.0056 - mean_squared_error: 0.0056 - val_loss: 0.0867 - val_mean_squared_error: 0.0867
Epoch 101/150
1051/1051 [==============================] - 0s 76us/step - loss: 0.0045 - mean_squared_error: 0.0045 - val_loss: 0.0868 - val_mean_squared_error: 0.0868
Epoch 102/150
1051/1051 [==============================] - 0s 74us/step - loss: 0.0052 - mean_squared_error: 0.0052 - val_loss: 0.0867 - val_mean_squared_error: 0.0867
Epoch 103/150
1051/1051 [==============================] - 0s 73us/step - loss: 0.0055 - mean_squared_error: 0.0055 - val_loss: 0.0896 - val_mean_squared_error: 0.0896
Epoch 104/150
1051/1051 [==============================] - 0s 71us/step - loss: 0.0044 - mean_squared_error: 0.0044 - val_loss: 0.0884 - val_mean_squared_error: 0.0884
Epoch 

Epoch 148/150
1051/1051 [==============================] - 0s 81us/step - loss: 0.0031 - mean_squared_error: 0.0031 - val_loss: 0.0891 - val_mean_squared_error: 0.0891
Epoch 149/150
1051/1051 [==============================] - 0s 78us/step - loss: 0.0045 - mean_squared_error: 0.0045 - val_loss: 0.0875 - val_mean_squared_error: 0.0875
Epoch 150/150
1051/1051 [==============================] - 0s 80us/step - loss: 0.0033 - mean_squared_error: 0.0033 - val_loss: 0.0986 - val_mean_squared_error: 0.0986


Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [31]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 32us/step


[0.007761146288389937, 0.007761146288389937]

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [32]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 41us/step


[0.09858456163706095, 0.09858456163706095]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [33]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer = 'Adam', loss = 'mse', metrics = ['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 364us/step - loss: 0.3805 - mean_squared_error: 0.3805 - val_loss: 0.1653 - val_mean_squared_error: 0.1653
Epoch 2/150
1051/1051 [==============================] - 0s 79us/step - loss: 0.1611 - mean_squared_error: 0.1611 - val_loss: 0.1334 - val_mean_squared_error: 0.1334
Epoch 3/150
1051/1051 [==============================] - 0s 89us/step - loss: 0.1131 - mean_squared_error: 0.1131 - val_loss: 0.1046 - val_mean_squared_error: 0.1046
Epoch 4/150
1051/1051 [==============================] - 0s 80us/step - loss: 0.0869 - mean_squared_error: 0.0869 - val_loss: 0.1022 - val_mean_squared_error: 0.1022
Epoch 5/150
1051/1051 [==============================] - 0s 81us/step - loss: 0.0747 - mean_squared_error: 0.0747 - val_loss: 0.1013 - val_mean_squared_error: 0.1013
Epoch 6/150
1051/1051 [==============================] - 0s 82us/step - loss: 0.0568 - mean_squared_error: 0.0568 - val_lo

Epoch 50/150
1051/1051 [==============================] - 0s 79us/step - loss: 0.0023 - mean_squared_error: 0.0023 - val_loss: 0.1019 - val_mean_squared_error: 0.1019
Epoch 51/150
1051/1051 [==============================] - 0s 84us/step - loss: 0.0017 - mean_squared_error: 0.0017 - val_loss: 0.0938 - val_mean_squared_error: 0.0938
Epoch 52/150
1051/1051 [==============================] - 0s 76us/step - loss: 0.0023 - mean_squared_error: 0.0023 - val_loss: 0.0998 - val_mean_squared_error: 0.0998
Epoch 53/150
1051/1051 [==============================] - 0s 81us/step - loss: 0.0022 - mean_squared_error: 0.0022 - val_loss: 0.0938 - val_mean_squared_error: 0.0938
Epoch 54/150
1051/1051 [==============================] - 0s 76us/step - loss: 0.0022 - mean_squared_error: 0.0022 - val_loss: 0.0946 - val_mean_squared_error: 0.0946
Epoch 55/150
1051/1051 [==============================] - 0s 75us/step - loss: 0.0018 - mean_squared_error: 0.0018 - val_loss: 0.0997 - val_mean_squared_error: 0.099

1051/1051 [==============================] - 0s 82us/step - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.0906 - val_mean_squared_error: 0.0906
Epoch 100/150
1051/1051 [==============================] - 0s 77us/step - loss: 0.0013 - mean_squared_error: 0.0013 - val_loss: 0.0924 - val_mean_squared_error: 0.0924
Epoch 101/150
1051/1051 [==============================] - 0s 88us/step - loss: 0.0011 - mean_squared_error: 0.0011 - val_loss: 0.0924 - val_mean_squared_error: 0.0924
Epoch 102/150
1051/1051 [==============================] - 0s 77us/step - loss: 0.0011 - mean_squared_error: 0.0011 - val_loss: 0.0913 - val_mean_squared_error: 0.0913
Epoch 103/150
1051/1051 [==============================] - 0s 77us/step - loss: 0.0011 - mean_squared_error: 0.0011 - val_loss: 0.0904 - val_mean_squared_error: 0.0904
Epoch 104/150
1051/1051 [==============================] - 0s 77us/step - loss: 0.0014 - mean_squared_error: 0.0014 - val_loss: 0.0929 - val_mean_squared_error: 0.0929
Epoch 

1051/1051 [==============================] - 0s 78us/step - loss: 0.0101 - mean_squared_error: 0.0101 - val_loss: 0.0935 - val_mean_squared_error: 0.0935
Epoch 148/150
1051/1051 [==============================] - 0s 75us/step - loss: 0.0088 - mean_squared_error: 0.0088 - val_loss: 0.0852 - val_mean_squared_error: 0.0852
Epoch 149/150
1051/1051 [==============================] - 0s 76us/step - loss: 0.0056 - mean_squared_error: 0.0056 - val_loss: 0.0927 - val_mean_squared_error: 0.0927
Epoch 150/150
1051/1051 [==============================] - 0s 77us/step - loss: 0.0052 - mean_squared_error: 0.0052 - val_loss: 0.0916 - val_mean_squared_error: 0.0916


Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [34]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 31us/step


[0.003493401781743575, 0.003493401781743575]

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [35]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 41us/step


[0.09157607177477825, 0.09157607177477825]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [36]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

146/146 [==============================] - 0s 45us/step


[0.2123309798436622, 0.2123309798436622]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [38]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

36210.8025530582

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.